In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.options.display.width = None
from glob import glob as g
import matplotlib as plot
import logging
import time
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
import random
pd.set_option('display.max_colwidth', 50)

import matplotlib.pyplot as plt

# Change the default settings of matplotlib
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 14
plt.rcParams['figure.dpi'] = 300  # Set the DPI to 100 globally

In [ ]:
#Read csv_file
finasteride = pd.read_csv('link')
tamsulosin = pd.read_csv('link')

In [ ]:
#take out the year value in the date_dispensed column and count the unique number of years
finasteride['year'] = finasteride['DATE_DISPENSED'].str[:4]
finasteride['year'].value_counts()

In [ ]:
#take out the year value in the date_dispensed column and count the unique number of years
tamsulosin['year'] = tamsulosin['DATE_DISPENSED'].str[:4]
tamsulosin['year'].value_counts()

In [ ]:
#Reassign the treament group
print(f'There are {len(tamsulosin)} patients in the tamsulosin group')
tamsulosin['Treatment'] = 1

print(f'There are {len(finasteride)} patients in the finasteride group')
finasteride['Treatment'] = 0

In [ ]:
#median of duration_year for finasteride
print(f"The median duration following-up of finasteride is :{finasteride['duration_year'].median()}") 
#median of duration_year for tamsulosin
print(f"The median duration following-up of finasteride is :{tamsulosin['duration_year'].median()}") 

In [ ]:
import matplotlib
print(matplotlib.get_cachedir())


In [ ]:
# Create a new column 'is_dead' based on 'DOD'
finasteride['is_dead'] = np.where(finasteride['DOD'].notnull() & finasteride['DATE_DISPENSED_insulin'].isnull(), 1, 0)
tamsulosin['is_dead'] = np.where(tamsulosin['DOD'].notnull() & tamsulosin['DATE_DISPENSED_insulin'].isnull(), 1, 0)

# Calculate the proportion of dead patients for each treatment
prop_dead_finasteride = finasteride['is_dead'].mean()
prop_dead_tamsulosin = tamsulosin['is_dead'].mean()

# Data to plot
proportions = [prop_dead_finasteride, prop_dead_tamsulosin]
treatment_labels = ['Finasteride', 'Tamsulosin']

# Create bar plot
plt.bar(treatment_labels, proportions, color=['red', 'blue'])
plt.xlabel('Treatment')
plt.ylabel('Proportion of Dead Patients')
plt.title('Proportion of Dead Patients by Treatment')
plt.show()

print(f'Death account for {round(prop_dead_finasteride,2)} of finasteride group.')
print(f'Death account for {round(prop_dead_tamsulosin,2)} of tamsulosin group.')

## Fix up data sets

### mapping ethnicity variables

In [ ]:
# Create a function to map original ethnicity codes to the new grouped categories
def group_and_map_ethnicity(df, ethnicity):
    # Create a function to map original ethnicity codes to the new grouped categories
    def map_ethnicity(ethnicity_code):
        if ethnicity_code in [11, 12, 10]:
            return 'European'
        elif ethnicity_code in [40, 41, 42, 43, 44, 51]:
            return 'Asian'
        elif ethnicity_code in [21, 31, 33, 35, 36, 34, 37]:
            return 'Indigenous'
        else:
            return 'Not_Stated/Other/Unknown'

    # Apply the map_ethnicity function to create a new ethnicity_grouped column
    df['ethnicity_grouped'] = df[ethnicity].apply(map_ethnicity)

    # Create dummy variables for the ethnicity_grouped categories
    ethnicity_grouped_dummies = pd.get_dummies(df['ethnicity_grouped'], prefix='ethnicity')

    # Convert the ethnicity_grouped variable to categorical format in pandas DataFrame
    df['ethnicity_grouped'] = df['ethnicity_grouped'].astype('category')

    # Concatenate the dummy variables with the original DataFrame
    df = pd.concat([df, ethnicity_grouped_dummies], axis=1)

    return df


In [ ]:
finasteride_df = finasteride.copy()
tamsulosin_df = tamsulosin.copy()

# Apply ethnicity function
finasteride_df= group_and_map_ethnicity(finasteride_df, 'ethnicity') 
tamsulosin_df= group_and_map_ethnicity(tamsulosin_df, 'ethnicity') 

## Summary table

In [ ]:
fin_tam_df = pd.concat([tamsulosin_df, finasteride_df], axis = 0)

In [ ]:
from scipy.stats import ttest_ind, ranksums, chi2_contingency

def summarize_variables(df, variables, grouped_variables):
    summary_rows = []
    # Create the summary row
    summary_row = {
    'Variable': 'Number of Patients',
    'Tamsulosin': len(df[df['Treatment'] == 1]),
    'Finasteride': len(df[df['Treatment'] == 0]),
    }
    # Append the summary row to the list
    summary_rows.append(summary_row)
    for category, var_list in grouped_variables.items():  
        summary_rows.append({'Variable': category, 'Tamsulosin': '', 'Finasteride': '', 'p-value': ''})      
        for variable in var_list:
            if variable in variables:
                row_data = {'Variable': f"  {variable}"}
                for treatment, treatment_name in [(1, 'Tamsulosin'), (0, 'Finasteride')]:
                    group = df[df['Treatment'] == treatment]
                    if variable in ['start_age', 'duration_year']:
                        mean = group[variable].mean()
                        std = group[variable].std()
                        value = f"{mean:.2f} ({std:.2f})"
                    elif variable in ['diabetes_duration','cci_score']:
                        median = group[variable].median()
                        iqr = group[variable].quantile(0.75) - group[variable].quantile(0.25)
                        value = f"{median:.2f} ({iqr:.2f})"
                    else:
                        count = group[variable].sum()
                        percentage = (count / len(group)) * 100
                        value = f"{count} ({percentage:.2f}%)"
                        
                    row_data[treatment_name] = value
                
                if category != 'Ethnicity':
                    # Conduct appropriate statistical test and get p-value
                    if variable in ['start_age', 'duration_year']:
                        _, p_value = ttest_ind(df[df['Treatment'] == 0][variable], df[df['Treatment'] == 1][variable])
                    elif variable in ['diabetes_duration','cci_score']:
                        _, p_value = ranksums(df[df['Treatment'] == 0][variable], df[df['Treatment'] == 1][variable])
                    else:
                        contingency_table = pd.crosstab(df['Treatment'], df[variable])
                        _, p_value, _, _ = chi2_contingency(contingency_table)
                    row_data['p-value'] = f"{p_value:.3f}"
                else:
                    row_data['p-value'] = ''

                summary_rows.append(row_data)
            
        if category == 'Ethnicity':
            row_data = {'Variable': f"  {category} (all)", 'Tamsulosin': '', 'Finasteride': '', 'p-value': ''}
            # Conduct chi-square test for the whole ethnicity variable
            contingency_table = pd.crosstab(df['Treatment'], [df[var] for var in var_list])
            _, p_value, _, _ = chi2_contingency(contingency_table)
            row_data['p-value'] = f"{p_value:.3f}"
            summary_rows.append(row_data)

    summary_table = pd.DataFrame(summary_rows)
    return summary_table

grouped_variables = {
    '': ['start_age', #'second_hospitalisation'
         'duration_year', 'cci_score', 'diabetes_duration'],
         'Diabetes medication': ['metformin','sulfonylureas','acarbose','GLP_1','DPP_4','SGLT_2'],
    'Other drug use': ['ACE_inhibitors', 'ARBs', 'Beta_blocker', 'corticosteroid', 'diuretics', 'statin'],
    'Ethnicity': ['ethnicity_European','ethnicity_Asian', 
                  'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown']
}
summary_variables = ['duration_year', #just to check- to be removed
   'start_age','diabetes_duration', 'cci_score',#'second_hospitalisation',
   'ACE_inhibitors', 'ARBs' ,'Beta_blocker', 'corticosteroid', 'diuretics', 'statin',
   'metformin','sulfonylureas','acarbose','GLP_1','DPP_4','SGLT_2',
   'ethnicity_European','ethnicity_Asian', 'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown',
]

In [ ]:
summarize_variables(fin_tam_df, summary_variables, grouped_variables)

#### Sumary table for weighted propensity

In [ ]:
#Calculate standardized difference
def calculate_standardized_diff(df, variable, for_means):
    tamsulosin_group = df[df['Treatment'] == 1][variable]
    finasteride_group = df[df['Treatment'] == 0][variable]
    
    if for_means: #for continuos variable
        mean_tamsulosin = tamsulosin_group.mean()
        mean_finasteride = finasteride_group.mean()
        
        pooled_std = np.sqrt((tamsulosin_group.std() ** 2 + finasteride_group.std() ** 2) / 2)
        
        std_diff = abs((mean_tamsulosin - mean_finasteride) / pooled_std)
    else: #for categorial variables
        prop_tamsulosin = tamsulosin_group.sum() / len(tamsulosin_group)
        prop_finasteride = finasteride_group.sum() / len(finasteride_group)
        
        std_diff = abs( 
            (prop_tamsulosin - prop_finasteride) / 
            np.sqrt( (prop_tamsulosin*(1 - prop_tamsulosin) + 
                      prop_finasteride*(1- prop_finasteride)) / 2) ) 
    
    return std_diff

In [ ]:
#Calculate standardized difference with weights
def calculate_weighted_standardized_diff(df, variable, weights, for_means):
    tamsulosin_group = df[df['Treatment'] == 1][[variable, weights]]
    finasteride_group = df[df['Treatment'] == 0][[variable, weights]]

    if for_means: #for continuous variable
        weighted_mean_tamsulosin = np.average(tamsulosin_group[variable], weights=tamsulosin_group[weights])
        weighted_mean_finasteride = np.average(finasteride_group[variable], weights=finasteride_group[weights])

        weighted_var_tamsulosin = np.average((tamsulosin_group[variable]-weighted_mean_tamsulosin)**2, weights=tamsulosin_group[weights])
        weighted_var_finasteride = np.average((finasteride_group[variable]-weighted_mean_finasteride)**2, weights=finasteride_group[weights])

        pooled_weighted_std = np.sqrt((weighted_var_tamsulosin + weighted_var_finasteride) / 2)

        std_diff = abs((weighted_mean_tamsulosin - weighted_mean_finasteride) / pooled_weighted_std)
    else: #for categorical variables
        weighted_prop_tamsulosin = tamsulosin_group.loc[tamsulosin_group[variable] == 1, weights].sum() / tamsulosin_group[weights].sum()
        weighted_prop_finasteride = finasteride_group.loc[finasteride_group[variable] == 1, weights].sum() / finasteride_group[weights].sum()

        std_diff = abs(
            (weighted_prop_tamsulosin - weighted_prop_finasteride) / 
            np.sqrt( (weighted_prop_tamsulosin*(1 - weighted_prop_tamsulosin) + 
                      weighted_prop_finasteride*(1 - weighted_prop_finasteride)) / 2) )

    return std_diff


In [ ]:
def summarize_variables_before_weighted(df, variables, grouped_variables):
    summary_rows = []
    #Create the summary row
    summary_row = {
    'Variable': 'Number of Patients',
    'Tamsulosin': len(df[df['Treatment'] == 1]),
    'Finasteride': len(df[df['Treatment'] == 0]),
    }
    # Append the summary row to the list
    summary_rows.append(summary_row)
    for category, var_list in grouped_variables.items():
        if category == 'Ethnicity':
            # Create a binary variable for dominant ethnicity (true = dominant ethnicity, false = other)
            dominant_ethnicity = df[['ethnicity_European', 'ethnicity_Asian', 'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown']].sum().idxmax()
            df['dominant_ethnicity'] = df[dominant_ethnicity].apply(lambda x: 1 if x else 0)

            # calculate overall standardized difference for 'Ethnicity'
            std_diff = calculate_standardized_diff(df, 'dominant_ethnicity', for_means=False)
            row_data = {'Variable': "  White vs others", 'Tamsulosin': '', 'Finasteride': '', 'Std Difference': f"{std_diff:.2f}"}
            summary_rows.append(row_data)
            
            # Drop the temporary dominant ethnicity column
            df.drop(columns='dominant_ethnicity', inplace=True)
            
        for variable in var_list:
            if variable in variables:
                row_data = {'Variable': f"    {variable}"}
                for treatment, treatment_name in [(1, 'Tamsulosin'), (0, 'Finasteride')]:
                    group = df[df['Treatment'] == treatment]
                    if variable in ['start_age', 'duration_year', 'diabetes_duration', 'cci_score']:
                        mean = group[variable].mean()
                        std = group[variable].std()
                        value = f"{mean:.2f} ({std:.2f})"
                    else:
                        count = group[variable].sum()
                        percentage = (count / len(group)) * 100
                        value = f"{count} ({percentage:.2f}%)"
                    
                    row_data[treatment_name] = value
                
                # calculate standardized difference for continuous variables
                if category != 'Ethnicity':
                    if variable in ['start_age', 'duration_year', 'diabetes_duration', 'cci_score']:
                        std_diff = calculate_standardized_diff(df, variable, for_means=True)
                    else: #for catgeorical variables
                        std_diff = calculate_standardized_diff(df, variable, for_means=False)
                        
                    row_data['Std Difference'] = f"{std_diff:.3f}"
                
                summary_rows.append(row_data)
            
    summary_table = pd.DataFrame(summary_rows)
    return summary_table


#### Summary variables after weighted

In [ ]:
# Summary the weights for each patient
def summarize_variables_after_weighted(df, variables, grouped_variables, weights_variable):
    summary_rows = []
    #Create the summary row
    summary_row = {
    'Variable': 'Number of Patients',
    'Tamsulosin': len(df[df['Treatment'] == 1]),
    'Finasteride': len(df[df['Treatment'] == 0]),
    }
    # Append the summary row to the list
    summary_rows.append(summary_row)
    for category, var_list in grouped_variables.items():
        if category == 'Ethnicity':
            # Create a binary variable for dominant ethnicity (true = dominant ethnicity, false = other)
            dominant_ethnicity = df[['ethnicity_European', 'ethnicity_Asian', 'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown']].sum().idxmax()
            df['dominant_ethnicity'] = df[dominant_ethnicity].apply(lambda x: 1 if x else 0)

            # calculate overall standardized difference for 'Ethnicity'
            std_diff = calculate_standardized_diff(df, 'dominant_ethnicity', for_means=False)
            row_data = {'Variable': "  White vs others", 'Tamsulosin': '', 'Finasteride': '', 'Std Difference': f"{std_diff:.2f}"}
            summary_rows.append(row_data)
            # standardize difference is independent of sample size. 
            # preference: https://support.sas.com/resources/papers/proceedings12/335-2012.pdf
            # Drop the temporary dominant ethnicity column
            df.drop(columns='dominant_ethnicity', inplace=True)
            
        for variable in var_list:
            if variable in variables:
                row_data = {'Variable': f"    {variable}"}
                for treatment, treatment_name in [(1, 'Tamsulosin'), (0, 'Finasteride')]:
                    group = df[df['Treatment'] == treatment]
                    if variable in ['start_age', 'duration_year', 'diabetes_duration', 'cci_score']:
                        weighted_mean = np.average(group[variable], weights=group[weights_variable])
                        weighted_std = np.sqrt(np.average((group[variable]-weighted_mean)**2, weights=group[weights_variable]))
                        value = f"{weighted_mean:.2f} ({weighted_std:.2f})"
                    else:
                        weighted_count = group.loc[group[variable] == 1, weights_variable].sum()
                        weighted_percentage = (weighted_count / group[weights_variable].sum()) * 100
                        value = f"{weighted_count:.0f} ({weighted_percentage:.2f}%)"
                    
                    row_data[treatment_name] = value
                
                # calculate standardized difference for continuous variables
                if category != 'Ethnicity':
                    if variable in ['start_age', 'duration_year', 'diabetes_duration', 'cci_score']:
                        std_diff = calculate_weighted_standardized_diff(df, variable,weights_variable, for_means=True)
                    else: #for catgeorical variables
                        std_diff = calculate_weighted_standardized_diff(df, variable,weights_variable, for_means=False)
                        
                    row_data['Std Difference'] = f"{std_diff:.2f}"
                
                summary_rows.append(row_data)
            
    summary_table = pd.DataFrame(summary_rows)
    return summary_table


## Functions

### Print out number of patients

In [ ]:
from lifelines import KaplanMeierFitter
#Produce summary table for incidence ratio
def incidence_ratio_table(df1, df2, df1_name, df2_name, interval):
    # Instantiate the KaplanMeierFitter
    kmf_df1 = KaplanMeierFitter()
    kmf_df2 = KaplanMeierFitter()

    # Fit the data into the model
    kmf_df1.fit(df1["duration_year"], df1["insulin_status"])
    kmf_df2.fit(df2["duration_year"], df2["insulin_status"])

    # Maximum duration_year value
    max_duration = max(df1['duration_year'].max(), df2['duration_year'].max())

    # Generate time_points list
    time_points = [round(i*interval, 2) for i in range(int(max_duration/interval)+1)]

    # Calculate cumulative incidence at specific time points
    cum_inc_df1 = 1 - kmf_df1.predict(time_points)
    cum_inc_df2 = 1 - kmf_df2.predict(time_points)

    # Calculate number at risk
    num_at_risk_df1 = (len(df1) - (cum_inc_df1 * len(df1))).astype(int)
    num_at_risk_df2 = (len(df2) - (cum_inc_df2 * len(df2))).astype(int)

    # Create number at risk table
    at_risk_table = pd.DataFrame({
        'Year': time_points,
        f'{df1_name} Number at risk': num_at_risk_df1.values,
        f'{df2_name} Number at risk': num_at_risk_df2.values
    })

    # Create cumulative incidence table
    ci_table = pd.DataFrame({
        'Year': time_points,
        f'{df1_name} Cumulative Incidence': cum_inc_df1.values,
        f'{df2_name} Cumulative Incidence': cum_inc_df2.values
    })
    #print number at risk table
    print(f'\n')
    print(at_risk_table.to_string(index = False))

    print(f"\n Cummulative incidence per year")
    print(ci_table.to_string(index=False))



### Kaplan-Meier curve

In [ ]:
#Function for Kaplan Meier curve and cummlative incidence
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def compute_incidence_rate(df, event_column, time_column):
    # Calculate the total number of events
    total_events = df[event_column].sum()

    # Calculate the total person-time at risk
    total_person_years = df[time_column].sum()

    # Calculate the incidence rate
    incidence_rate = round(((total_events / total_person_years)* 10000), 2) #per 10000 person-years

    # Calculate the 95% confidence interval
    sqrt_events = np.sqrt(total_events)
    ci_lower = round(((total_events - 1.96 * sqrt_events) / total_person_years) *10000, 2)
    ci_upper = round(((total_events + 1.96 * sqrt_events) / total_person_years)* 10000, 2)

    return incidence_rate, ci_lower, ci_upper

def cumulative_incidence_and_km_plot(df1, df2, df1_name, df2_name, table_name):
    # fit Kaplan-Meier curves and calculate log-rank p-value
    kmf_fin = KaplanMeierFitter()
    kmf_tam = KaplanMeierFitter()

    # fit dataframes curves
    kmf_fin.fit(df1['duration_year'], event_observed=df1['insulin_status'], label=df1_name)
    kmf_tam.fit(df2['duration_year'], event_observed=df2['insulin_status'], label=df2_name)

    # plot Cumulative Incidence curves
    # fig, ax2 = plt.subplots(figsize=(12, 6), dpi=300)  # set dpi when creating the figure
    ax2 = (1 - kmf_fin.survival_function_).plot(label=f'{df1_name} (Cumulative Incidence)', color='blue', linestyle='-')
    (1 - kmf_tam.survival_function_).plot(ax=ax2, label=f'{df2_name} (Cumulative Incidence)', color='red', linestyle='-')
    # Set the figure size
    ax2.figure.set_size_inches(12, 6)
    #ax2.set_title('Cumulative Incidence Over Time')
    ax2.set_xlabel('Duration of following-up (years)')
    ax2.set_ylabel('Cumulative Incidence of insulin prescriptions')
    ax2.legend()


    # calculate and print log-rank p-value
    results = logrank_test(df1['duration_year'], df2['duration_year'], df1['insulin_status'], df2['insulin_status'], alpha=.99)
    print('Log-Rank Test p-value:', results.p_value)

    # create a summary table
    summary_table = pd.DataFrame({'Treatment Group': [df1_name, df2_name],
                                  'Insulin Status 0': [df1['insulin_status'].value_counts()[0], df2['insulin_status'].value_counts()[0]],
                                  'Insulin Status 1': [df1['insulin_status'].value_counts()[1], df2['insulin_status'].value_counts()[1]],
                                  'p-value': [results.p_value, '']})
    print("\nSummary Table:")
    print(summary_table)

    # # print number at risk
    # print("\nNumber at risk at each time point (years):")
    time_points = [1, 2, 3, 4, 5, 6, 7, 8, 9]  # You can change these time points to your desired values

    # calculate cumulative incidence at specific time points (in years)
    cum_inc_fin = 1 - kmf_fin.predict(time_points)
    cum_inc_tam = 1 - kmf_tam.predict(time_points)
    # calculate number at risk
    num_at_risk_fin = (len(df1) - (cum_inc_fin * len(df1))).astype(int).rename(f"{df1_name} at risk")
    num_at_risk_tam = (len(df2) - (cum_inc_tam * len(df2))).astype(int).rename(f"{df2_name} at risk")
    at_risk_table = pd.DataFrame({'Year': time_points,
                                f'{df1_name} Number at risk': num_at_risk_fin,
                                f'{df2_name} Number at risk': num_at_risk_tam})
    #print number at risk table
    print(f'\n')
    print(at_risk_table.to_string(index = False))
    #print cummulative incidence table
    ci_table = pd.DataFrame({'Year': time_points,
                             f'{df1_name} Cumulative Incidence': cum_inc_fin,
                             f'{df2_name} Cumulative Incidence': cum_inc_tam})
    print(f"\n{table_name}:")
    print(ci_table.to_string(index=False))

    # Compute the incidence rate and confidence interval for df1 and df2
    incidence_rate_df1, ci_lower_df1, ci_upper_df1 = compute_incidence_rate(df1, 'insulin_status', 'duration_year')
    incidence_rate_df2, ci_lower_df2, ci_upper_df2 = compute_incidence_rate(df2, 'insulin_status', 'duration_year')

    print(f"Incidence rate for {df1_name}: {incidence_rate_df1} events per 10000 person-year (95% CI: {ci_lower_df1} to {ci_upper_df1})")
    print(f"Incidence rate for {df2_name}: {incidence_rate_df2} events per 10000 person-year (95% CI: {ci_lower_df2} to {ci_upper_df2})")


    

### Plot functions

In [ ]:
#Create Kernel plot for visualisation
def plot_kde(df1, df2, column):
    df = pd.concat([df1, df2], axis = 0 )
    tamsulosin = df[df['Treatment'] == 1][f'{column}']
    finasteride = df[df['Treatment'] == 0][f'{column}']

    plt.figure(figsize=(10, 6))
    sns.kdeplot(tamsulosin, shade=True, color='blue', label='Tamsulosin')
    sns.kdeplot(finasteride, shade=True, color='red', label='Finasteride')

    plt.title(f'{column} Density Plots for Tamsulosin and Finasteride')
    plt.xlabel(f'{column}')
    plt.ylabel('Density')
    plt.legend()

    plt.show()

In [ ]:
#box plot
def plot_box(df1, df2, column):
    tamsulosin = df1[df1['Treatment'] == 1][f'{column}']
    finasteride = df2[df2['Treatment'] == 0][f'{column}']

    plt.figure(figsize=(10, 6))
    # Customize font size and font family
    font = {'family': 'Times New Roman', 'size': 16}
    plt.rc('font', **font)

    # Create a boxplot for each group
    plt.boxplot([tamsulosin, finasteride], labels=['Tamsulosin', 'Finasteride'])

    plt.title(f'{column} Box Plots for Tamsulosin and Finasteride')
    plt.xlabel('Group')
    plt.ylabel(f'{column}')

    plt.show()

#weighted box_plot
def plot_box_weighted(df1, df2, column, weights):
    tamsulosin = df1[df1['Treatment'] == 1][[column, weights]]
    finasteride = df2[df2['Treatment'] == 0][[column, weights]]

    # Resample data according to weights
    tamsulosin_resampled = np.repeat(tamsulosin[column], tamsulosin[weights].astype(int))
    finasteride_resampled = np.repeat(finasteride[column], finasteride[weights].astype(int))

    plt.figure(figsize=(10, 6))

    # Create a boxplot for each group
    plt.boxplot([tamsulosin_resampled, finasteride_resampled], labels=['Tamsulosin', 'Finasteride'])

    plt.title(f'{column} Box Plots for Tamsulosin and Finasteride after weighted')
    plt.xlabel('Group')
    plt.ylabel(f'{column}')

    plt.show()


In [ ]:
def plot_box_and_weighted_box(df1, df2, column, weights, column_name):
    # Create two subplots: one for unweighted data and one for weighted data
    fig, axs = plt.subplots(1, 2, figsize=(10, 6), dpi=300)
    
    # Unweighted box plot
    tamsulosin = df1[df1['Treatment'] == 1][column]
    finasteride = df2[df2['Treatment'] == 0][column]
    axs[0].boxplot([tamsulosin, finasteride], labels=['Tamsulosin', 'Finasteride'])
    axs[0].set_title('Box plot in unweighted sample')
    # axs[0].set_xlabel('Group')
    axs[0].set_ylabel(f'{column_name}')
    axs[0].grid(True)  # Add grid-lines

    # Weighted box plot
    tamsulosin_weighted = df1[df1['Treatment'] == 1][[column, weights]]
    finasteride_weighted = df2[df2['Treatment'] == 0][[column, weights]]
    tamsulosin_resampled = np.repeat(tamsulosin_weighted[column], tamsulosin_weighted[weights].astype(int))
    finasteride_resampled = np.repeat(finasteride_weighted[column], finasteride_weighted[weights].astype(int))
    axs[1].boxplot([tamsulosin_resampled, finasteride_resampled], labels=['Tamsulosin', 'Finasteride'])
    axs[1].set_title('Box plot in weighted sample')
    # axs[1].set_xlabel('Group')
    axs[1].set_ylabel(f'{column_name}')
    axs[1].grid(True)  # Add grid-lines

    # Adjust layout
    plt.tight_layout()
    plt.show()


#### cummulative distribution function plot

In [ ]:
def plot_ecdf(df1, df2, column):
    """
    Compute and plot ECDFs for two unweighted datasets.

    Parameters
    ----------
    df1, df2: DataFrame
        DataFrames containing the data.
    column: str
        The column name of the data points for which the ECDF will be computed.
    """

    # Number of data points
    n1 = len(df1[column])
    n2 = len(df2[column])

    # x-data for the ECDF: sorted data
    x1 = np.sort(df1[column])
    x2 = np.sort(df2[column])

    # y-data for the ECDF: evenly spaced sequence from 1/n to 1
    y1 = np.arange(1, n1+1) / n1
    y2 = np.arange(1, n2+1) / n2

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(x1, y1, label='dataset_1')
    plt.plot(x2, y2, label='dataset_2')
    plt.xlabel('Value')
    plt.ylabel(f'Cumulative Probability of {column}')
    plt.title('Empirical Cumulative Distribution Functions')
    plt.legend()
    plt.show()


In [ ]:
def plot_weighted_ecdf(df1, df2, column, weights, column_name):
    """
    Compute and plot ECDFs for two weighted datasets.

    Parameters
    ----------
    df1, df2: DataFrame
        DataFrames containing the data and weights.
    column: str
        The column name of the data points for which the ECDF will be computed.
    weights: str
        The column name of the weights.
    """

    # Resample data according to weights
    resampled_data1 = np.repeat(df1[column], df1[weights].astype(int))
    resampled_data2 = np.repeat(df2[column], df2[weights].astype(int))

    # Sort data and compute cumulative probabilities
    x1 = np.sort(resampled_data1)
    y1 = np.arange(1, len(x1)+1) / len(x1)

    x2 = np.sort(resampled_data2)
    y2 = np.arange(1, len(x2)+1) / len(x2)

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(x1, y1, label='Dataset 1')
    plt.plot(x2, y2, label='Dataset 2')
    plt.xlabel(column_name)
    plt.ylabel(f'Cumulative Probability of {column_name}')
    plt.title('Cumulative Distribution in weighted poplation')
    plt.legend()
    plt.show()


#### cummulative distribution plot

In [ ]:
def plot_ecdf_and_weighted_ecdf(df1, df2, column, weights, column_name):
    """
    Compute and plot ECDFs for unweighted and weighted datasets in two subplots.

    Parameters
    ----------
    df1, df2: DataFrame
        DataFrames containing the data and weights.
    column: str
        The column name of the data points for which the ECDFs will be computed.
    weights: str
        The column name of the weights.
    """

    # Number of data points
    n1 = len(df1[column])
    n2 = len(df2[column])

    # x-data for the ECDF: sorted data
    x1 = np.sort(df1[column])
    x2 = np.sort(df2[column])

    # y-data for the ECDF: evenly spaced sequence from 1/n to 1
    y1 = np.arange(1, n1+1) / n1
    y2 = np.arange(1, n2+1) / n2

    # Resample data according to weights
    resampled_data1 = np.repeat(df1[column], df1[weights].astype(int))
    resampled_data2 = np.repeat(df2[column], df2[weights].astype(int))

    # Sort data and compute cumulative probabilities for weighted data
    x1_weighted = np.sort(resampled_data1)
    y1_weighted = np.arange(1, len(x1_weighted)+1) / len(x1_weighted)

    x2_weighted = np.sort(resampled_data2)
    y2_weighted = np.arange(1, len(x2_weighted)+1) / len(x2_weighted)

    # Create the figure and subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 6), dpi=300)
    
    # Plot unweighted ECDFs
    axs[0].plot(x1, y1, label='Tamsulosin') #where treatment = 1
    axs[0].plot(x2, y2, label='Finasteride')  #where treatment = 0
    axs[0].set_xlabel(column_name)
    axs[0].set_ylabel('Proportion <= x')
    axs[0].set_title('Cumulative Distribution in unweighted poplation')
    axs[0].legend()

    # Plot weighted ECDFs
    axs[1].plot(x1_weighted, y1_weighted, label='Tamsulosin')
    axs[1].plot(x2_weighted, y2_weighted, label='Finasteride')
    axs[1].set_xlabel(column_name)
    axs[1].set_ylabel('Proportion <= x')
    axs[1].set_title('Cumulative Distribution in weighted poplation')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

### Lag-time function

In [ ]:
def apply_lag_time(df, lag_duration):
    # Create a copy of the dataframe to avoid modifying the original one
    df_lag = df.copy()

    # Identify the rows where the event occurred before the lag time
    early_event_mask = df_lag['duration_year'] < lag_duration

    # For these rows, set the event status to 0 (non-case)
    df_lag.loc[early_event_mask, 'insulin_status'] = 0

    # Return the modified dataframe
    return df_lag


#### Test case for lag-time function

In [ ]:
import pandas as pd

def test_apply_lag_time():
    # Create a test dataframe
    data = {
        'insulin_status': [0, 1, 1, 0, 1],
        'duration_year': [1, 2, 3, 4, 5]
    }
    df = pd.DataFrame(data)

    # Apply the lag time
    df_lag = apply_lag_time(df, 3)

    # Check the output
    assert df_lag['insulin_status'].tolist() == [0, 0, 1, 0, 1], "Test failed!"
    assert df_lag['duration_year'].tolist() == [1, 2, 3, 4, 5], "Test failed!"

# Run the test
test_apply_lag_time()


### Visualisation

In [ ]:
finasteride_df['duration_year'].describe()

In [ ]:
tamsulosin_df['duration_year'].describe()

In [ ]:
plot_kde(finasteride_df, tamsulosin_df, 'diabetes_duration')

In [ ]:
plot_kde(tamsulosin_df, finasteride_df, 'duration_year')

In [ ]:
plot_kde(tamsulosin_df, finasteride_df, 'start_age')

In [ ]:
plot_kde(tamsulosin_df.query('insulin_status == 1'), finasteride_df.query('insulin_status == 1'), 'duration_year')

In [ ]:
plot_kde(tamsulosin_df, finasteride_df, 'cci_score')

## Multi-variable Cox-proportional hazard model

In [ ]:
unadjusted_variables = ['Treatment', 'insulin_status', 'duration_year']

In [ ]:
variables = ['Treatment',
       'ACE_inhibitors', 'ARBs' ,'Beta_blocker',   'diuretics', 'statin',
       #'corticosteroid', 
       #'second_hospitalisation', 
       'insulin_status','duration_year',
       'start_age', 'cci_score', 'diabetes_duration',
       'metformin', 'sulfonylureas',
       'ethnicity_Asian', 'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown']

In [ ]:
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
from scipy.stats import chi2_contingency, f_oneway, kruskal
from scipy.stats import ranksums

def Cox_proportional_model(df1, df2, df1_name, df2_name, variables):
    
    # Merge the two data frames
    df = pd.concat([df1, df2], axis=0)
    # Run the Cox proportional hazard model
    np.random.seed(42)

    cph = CoxPHFitter()
    cph.fit(df[variables],
        duration_col='duration_year', event_col='insulin_status')

    summary_table = cph.summary
    
    treatment_labels = [df1_name, df2_name]
    for i, label in enumerate(treatment_labels):
        mask = (df['Treatment'] == i)
        patients = mask.sum()
        event_count = df[mask]['insulin_status'].sum()
        no_event_count = patients - event_count
        print(f'Treatment {i} ({label}): {patients} patients, {event_count} events, {no_event_count} censored')

    fig, ax = plt.subplots(figsize=(10, 6), dpi = 300)

    # Plot Cox-proportional hazard
    cph.plot_partial_effects_on_outcome('Treatment', [0, 1], cmap='coolwarm', ax=ax)

    # Access the lines and change their colors
    ax.get_lines()[0].set_color('blue')
    ax.get_lines()[1].set_color('red')

    ax.set_xlabel('Duration of following-up (years')
    ax.set_ylabel('Probability of receiving insulin prescription')
    # ax.set_title(f'Survival curves of {df1_name} versus {df2_name}')
    ax.legend(labels=treatment_labels)

    plt.show()

    return summary_table


### Propensity score

In [ ]:
from sklearn.linear_model import LogisticRegression

covariates = ['ACE_inhibitors', 'ARBs' ,'Beta_blocker', 'diuretics', 'statin', #medications
             #'corticosteroid',
             #'second_hospitalisation', 
             'metformin', 'sulfonylureas', #T2DMmedications
             'start_age',
             'cci_score', 'diabetes_duration', 
             'ethnicity_Asian', 'ethnicity_Indigenous','ethnicity_Not_Stated/Other/Unknown']

def estimate_propensity_scores(df1, df2):
    # Merge the two dataframes
    df = pd.concat([df1, df2], axis=0)

    # Define the predictors and the outcome variable
    X = df[covariates]
    y = df['Treatment']

    # Estimate propensity scores using logistic regression
    logistic_regression = LogisticRegression(solver='lbfgs', max_iter=1000)
    logistic_regression.fit(X, y)
    df['propensity_score'] = logistic_regression.predict_proba(X)[:, 1]
    # Trimming the weights
    lower_bound, upper_bound = df['propensity_score'].quantile([0.01, 0.99])  # Modify this line according to your chosen percentiles
    df['propensity_score'] = np.where(df['propensity_score'] < lower_bound, lower_bound,
                         np.where(df['propensity_score'] > upper_bound, upper_bound, df['propensity_score']))


    return df

### Kaplan Meier-curve after propensity score weighting

In [ ]:
def cumulative_incidence_weighted(df, df1_name, df2_name):
    # fit Kaplan-Meier curves and calculate log-rank p-value
    kmf_tam = KaplanMeierFitter()
    kmf_fin = KaplanMeierFitter()

    #df_1 is finasteride, df_2 is tamsulosin
    df1 = df[df['Treatment'] == 0]
    df2 = df[df['Treatment'] == 1]

    # fit dataframes curves
    kmf_tam.fit(df1['duration_year'], event_observed=df1['insulin_status'], label=df1_name, weights=df1['ipw'])
    kmf_fin.fit(df2['duration_year'], event_observed=df2['insulin_status'], label=df2_name, weights=df2['ipw'])

    # plot Cumulative Incidence curves
    ax2 = (1 - kmf_fin.survival_function_).plot(label=f'{df1_name} (Cumulative Incidence)', color='red', linestyle='--')
    (1 - kmf_tam.survival_function_).plot(ax=ax2, label=f'{df2_name} (Cumulative Incidence)', color='blue', linestyle='-')
    ax2.set_title('Cumulative Incidence Over Time')
    ax2.set_xlabel('Time (years)')
    ax2.set_ylabel('Cumulative Incidence')
    ax2.legend()

    # calculate and print log-rank p-value
    results = logrank_test(df1['duration_year'], df2['duration_year'], df1['insulin_status'], df2['insulin_status'], alpha=.99)
    print('Log-Rank Test p-value:', results.p_value)

### Inverese probability weighting

In [ ]:
def cox_ipw_and_graphs(df, df1_control_name,df2_treatment_name):

   # Calculate inverse probability weights
    df['ipw'] = np.where(df['Treatment'] == 1, 1/df['propensity_score'], 1/(1 - df['propensity_score']))

    # Weight trimming at 1st and 99th percentile
    lower_bound = df['ipw'].quantile(0.01)
    upper_bound = df['ipw'].quantile(0.99)
    df['ipw'] = df['ipw'].clip(lower=lower_bound, upper=upper_bound)
    
    #Output summary table
    print(summarize_variables_after_weighted(df,variables, grouped_variables, 'ipw'))
    
    #boxplot for weighted data to check distribution
    plot_box_and_weighted_box(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'cci_score', 'ipw', 'CCI score')
    plot_box_and_weighted_box(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'diabetes_duration', 'ipw', 'Diabetes duration')
    plot_box_and_weighted_box(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'start_age', 'ipw', 'Age')
    
    #plot ecdf for weighted data
    plot_ecdf_and_weighted_ecdf(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'cci_score', 'ipw', 'CCI score')
    plot_ecdf_and_weighted_ecdf(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'diabetes_duration', 'ipw', 'Diabetes duration')
    plot_ecdf_and_weighted_ecdf(df[df['Treatment'] == 1], df[df['Treatment'] == 0], 'start_age', 'ipw', 'Age')
    
    # Fit the Cox proportional hazard model with IPW
    cph = CoxPHFitter()
    cph.fit(df[['Treatment', 
                'duration_year','insulin_status', 'ipw']], 
            duration_col='duration_year', event_col='insulin_status', weights_col='ipw',
            robust = True)
    
    #Counting patients in each treatment group
    treatment_labels = [df1_control_name, df2_treatment_name]
    for i, label in enumerate(treatment_labels):
        mask = (df['Treatment'] == i)
        patients = mask.sum()
        event_count = df[mask]['insulin_status'].sum()
        no_event_count = patients - event_count

        # Calculate the weighted event count
        weighted_event_count = (df[mask]['insulin_status'] * df[mask]['ipw']).sum()

        # Calculate the total number of patients in the weighted population
        weighted_patient_count = df[mask]['ipw'].sum()

        print(f'''Treatment {i} ({label}): {patients} patients, {event_count} events, {no_event_count} censored, 
              {round(weighted_event_count,0)} weighted events, {round(weighted_patient_count,0)} total patients in weighted population''')
                    
    
    # Plot Cox-proportional hazard
    fig, ax = plt.subplots(figsize=(10, 6))
    cph.plot_partial_effects_on_outcome('Treatment', [0, 1], cmap='coolwarm', ax=ax)

    ax.set_xlabel('Duration of following-up (years)')
    ax.set_ylabel('Probability of receiving insulin prescriptions')
    # ax.set_title(f'Survival curves of {df1_control_name} versus {df2_treatment_name}')
    ax.legend(labels=[df1_control_name, df2_treatment_name])

    plt.show()

    return cph.summary

In [ ]:
def cox_ipw(df, df1_control_name,df2_treatment_name):

   # Calculate inverse probability weights
    df['ipw'] = np.where(df['Treatment'] == 1, 1/df['propensity_score'], 1/(1 - df['propensity_score']))

    # Weight trimming at 1st and 99th percentile
    lower_bound = df['ipw'].quantile(0.01)
    upper_bound = df['ipw'].quantile(0.99)
    df['ipw'] = df['ipw'].clip(lower=lower_bound, upper=upper_bound)
    
    #Output summary table
    print(summarize_variables_after_weighted(df,variables, grouped_variables, 'ipw'))
    
     # Fit the Cox proportional hazard model with IPW
    cph = CoxPHFitter()
    cph.fit(df[['Treatment', 
                'duration_year','insulin_status', 'ipw']], 
            duration_col='duration_year', event_col='insulin_status', weights_col='ipw',
            robust = True)
    
    
    #Counting patients in each treatment group
    treatment_labels = [df1_control_name, df2_treatment_name]
    for i, label in enumerate(treatment_labels):
        mask = (df['Treatment'] == i)
        patients = mask.sum()
        event_count = df[mask]['insulin_status'].sum()
        no_event_count = patients - event_count

        # Calculate the weighted event count
        weighted_event_count = (df[mask]['insulin_status'] * df[mask]['ipw']).sum()

        # Calculate the total number of patients in the weighted population
        weighted_patient_count = df[mask]['ipw'].sum()

        print(f'''Treatment {i} ({label}): {patients} patients, {event_count} events, {no_event_count} censored, 
              {round(weighted_event_count,0)} weighted events, {round(weighted_patient_count,0)} total patients in weighted population''')
                
    
    fig, ax = plt.subplots(figsize=(10, 6), dpi = 300)

    # Plot Cox-proportional hazard
    cph.plot_partial_effects_on_outcome('Treatment', [0, 1], cmap='coolwarm', ax=ax)

    # Access the lines and change their colors
    ax.get_lines()[0].set_color('blue')
    ax.get_lines()[1].set_color('red')

    ax.set_xlabel('Duration of following-up (years)')
    ax.set_ylabel('Probability of receiving insulin prescriptions')
    # ax.set_title(f'Survival curves of {df1_control_name} versus {df2_treatment_name}')
    ax.legend(labels=[df1_control_name, df2_treatment_name])

    plt.show()

    return cph.summary

In [ ]:
def cox_ipw_strata(df, df1_control_name,df2_treatment_name):

   # Calculate inverse probability weights
    df['ipw'] = np.where(df['Treatment'] == 1, 1/df['propensity_score'], 1/(1 - df['propensity_score']))

    # Weight trimming at 1st and 99th percentile
    lower_bound = df['ipw'].quantile(0.01)
    upper_bound = df['ipw'].quantile(0.99)
    df['ipw'] = df['ipw'].clip(lower=lower_bound, upper=upper_bound)
    
    #Output summary table
    print(summarize_variables_after_weighted(df,variables, grouped_variables, 'ipw'))
    
    # Fit the Cox proportional hazard model with IPW
    cph = CoxPHFitter()
    cph.fit(df[['Treatment', 
                'duration_year','insulin_status', 'ipw']], 
            duration_col='duration_year', event_col='insulin_status', weights_col='ipw',
            robust = True)
    
    
    #Counting patients in each treatment group
    treatment_labels = [df1_control_name, df2_treatment_name]
    for i, label in enumerate(treatment_labels):
        mask = (df['Treatment'] == i)
        patients = mask.sum()
        event_count = df[mask]['insulin_status'].sum()
        no_event_count = patients - event_count

        # Calculate the weighted event count
        weighted_event_count = (df[mask]['insulin_status'] * df[mask]['ipw']).sum()

        # Calculate the total number of patients in the weighted population
        weighted_patient_count = df[mask]['ipw'].sum()

        print(f'''Treatment {i} ({label}): {patients} patients, {event_count} events, {no_event_count} censored, 
              {round(weighted_event_count,0)} weighted events, {round(weighted_patient_count,0)} total patients in weighted population''')
                
    
    fig, ax = plt.subplots(figsize=(10, 6), dpi = 300)

    # Plot Cox-proportional hazard
    cph.plot_partial_effects_on_outcome('Treatment', [0, 1], cmap='coolwarm', ax=ax)

    # Access the lines and change their colors
    ax.get_lines()[0].set_color('blue')
    ax.get_lines()[1].set_color('red')

    ax.set_xlabel('Duration of following-up (years)')
    ax.set_ylabel('Probability of receiving insulin prescriptions')
    # ax.set_title(f'Survival curves of {df1_control_name} versus {df2_treatment_name}')
    ax.legend(labels=[df1_control_name, df2_treatment_name])

    plt.show()

    return cph.summary

In [ ]:
def check_assumption_cox(df1, df2, df1_name, df2_name, variables):
    
    # Merge the two data frames
    df = pd.concat([df1, df2], axis=0)
    # Run the Cox proportional hazard model
    np.random.seed(42)

    cph = CoxPHFitter()
    cph.fit(df[variables],
        duration_col='duration_year', event_col='insulin_status')

    summary_table = cph.summary

    # Check the proportional hazard assumption
    df_check_assumption = df.drop('new_enc_nhi', axis=1)
    df_check_assumption= df_check_assumption.reset_index(drop=True)
    # Check the proportional hazards assumption
    # Select only the columns you're interested in
    columns_of_interest = variables
    df_subset = df_check_assumption[columns_of_interest].copy()
    cph.check_assumptions(df_subset, p_value_threshold=0.05, show_plots=True, 
                       )
        # Print the summary table
    treatment_labels = [df1_name, df2_name]
    for i, label in enumerate(treatment_labels):
        mask = (df['Treatment'] == i)
        patients = mask.sum()
        event_count = df[mask]['insulin_status'].sum()
        no_event_count = patients - event_count
        print(f'Treatment {i} ({label}): {patients} patients, {event_count} events, {no_event_count} censored')

    fig, ax = plt.subplots(figsize=(10, 6))

    cph.plot_partial_effects_on_outcome('Treatment', [0, 1], cmap='coolwarm', ax=ax)

    ax.set_xlabel('Duration (years)')
    ax.set_ylabel('Survival probability')
    ax.set_title(f'Survival curves of {df1_name} versus {df2_name}')
    ax.legend(labels=treatment_labels)

    plt.show()

    return summary_table

### Stratification by cci score

In [ ]:
variables_strata_cci = ['Treatment',
       'ACE_inhibitors', 'ARBs' ,'Beta_blocker',   'diuretics', 'statin',
       #'corticosteroid', 
       #'second_hospitalisation', 
       'metformin', 'sulfonylureas',
       'insulin_status','duration_year',
       'start_age', #'cci_score', 
       'diabetes_duration',
       'ethnicity_Asian', 'ethnicity_Indigenous', 'ethnicity_Not_Stated/Other/Unknown']
#percentile strata
def run_cox_models_percentile_strata(df1, df2, df1_name, df2_name):
    # Merge the two dataframes
    aggregate_df = pd.concat([df1, df2], axis=0)

    # Calculate the percentile values for cci_score
    median_cci = aggregate_df['cci_score'].quantile(0.5)
    tertile_1_cci = aggregate_df['cci_score'].quantile(1/3)
    tertile_2_cci = aggregate_df['cci_score'].quantile(2/3)

    print(f"Median CCI score: {median_cci:.2f}")

    # Define the strata based on cci_score percentiles
    strata = {
        'scenario_1': {
            'lower': (0, median_cci),
            'higher': (median_cci, np.inf)
        },
        # 'scenario_2': {
        #     'tertile_1': (0, tertile_1_cci),
        #     'tertile_2': (tertile_1_cci, tertile_2_cci),
        #     'tertile_3': (tertile_2_cci, np.inf)
        # }
    }

    # Loop through the scenarios and strata and run the Cox proportional hazard model for each stratum
    for scenario, strata_ranges in strata.items():
        print(f"{scenario.capitalize()}:")
            # add this before your loop
        all_summary_tables = []
        for stratum_name, cci_range in strata_ranges.items():
            print(f"{stratum_name.capitalize()} stratum:")

            # Filter the DataFrames based on the cci_score range for the current stratum
            filtered_df1 = df1.loc[(df1['cci_score'] > cci_range[0]) & (df1['cci_score'] <= cci_range[1])]
            filtered_df2 = df2.loc[(df2['cci_score'] > cci_range[0]) & (df2['cci_score'] <= cci_range[1])]

            #calculate propensity score for each stratum
            total_filtered_df_propensity_score = estimate_propensity_scores(filtered_df1, filtered_df2)

            # Run the Cox proportional hazard model for the filtered DataFrames
            test_table_name = f"{stratum_name.capitalize()} Stratum ({scenario}) - {df1_name} vs {df2_name}"
            summary_table = cox_ipw_strata(total_filtered_df_propensity_score, df1_name, df2_name)
            
            all_summary_tables.append(summary_table)
            
    # add this at the end of your function
    return all_summary_tables

## Primary analysis

In [ ]:
#Kaplan Meier
cumulative_incidence_and_km_plot(finasteride_df,tamsulosin_df, 'Finasteride','Tamsulosin','Cumulative Incidence')

In [ ]:
check_assumption_cox(finasteride_df, tamsulosin_df, 'Finasteride', 'Tamsulosin', variables)

In [ ]:
#diabetes duration visualisation
finasteride_df['diabetes_duration'].hist(bins=20, alpha=0.5, label='Finasteride')

In [ ]:
tamsulosin['diabetes_duration'].hist(bins=20, alpha=0.5, label='Tamsulosin')

#### Unadjusted Cox model

In [ ]:
Cox_proportional_model(tamsulosin_df, finasteride_df, 'Finasteride_unadjusted', 'Tamsulosin_unadjusted', unadjusted_variables)

### Adjusted Hazard ratio

In [ ]:
Cox_proportional_model(tamsulosin_df, finasteride_df, 'Finasteride', 'Tamsulosin', variables)

### IPTW of primary analysis

In [ ]:
summarize_variables_before_weighted(fin_tam_df, variables, grouped_variables)

In [ ]:
#calculate the median of tamsulosin and finasteride duration_year variable
print(f"Median of duration of finasteride is {finasteride_df['duration_year'].median()}")
print(f"Median of duration of tamsulosin is {tamsulosin['duration_year'].median()}") 

In [ ]:
#propensity score for primary analysis
propensity_score_primary_df = estimate_propensity_scores(tamsulosin_df, finasteride_df)

In [ ]:
propensity_score_primary_df

### Check assumption IPTW

In [ ]:
def check_assumption_IPTW(df, df1_name, df2_name, variables):
    # Calculate inverse probability weights
    df['ipw'] = np.where(df['Treatment'] == 1, 1/df['propensity_score'], 1/(1 - df['propensity_score']))

    # Weight trimming at 1st and 99th percentile
    lower_bound = df['ipw'].quantile(0.01)
    upper_bound = df['ipw'].quantile(0.99)
    df['ipw'] = df['ipw'].clip(lower=lower_bound, upper=upper_bound)
    
    cph = CoxPHFitter()
    cph.fit(df[variables],
        duration_col='duration_year', event_col='insulin_status', weights_col='ipw')

    summary_table = cph.summary

    # Check the proportional hazard assumption
    df_check_assumption = df.drop('new_enc_nhi', axis=1)
    df_check_assumption= df_check_assumption.reset_index(drop=True)
    # Check the proportional hazards assumption
    # Select only the columns you're interested in
    columns_of_interest = variables
    df_subset = df_check_assumption[columns_of_interest].copy()
    cph.check_assumptions(df_subset, p_value_threshold=0.05, show_plots=True, 
                       )

    return summary_table

In [ ]:
check_assumption_IPTW(propensity_score_primary_df, 'Finasteride', 'Tamsulosin', variables)

In [ ]:
cox_ipw_and_graphs(propensity_score_primary_df, 'Finasteride', 'Tamsulosin')

In [ ]:
cumulative_incidence_weighted(propensity_score_primary_df, 'Finasteride', 'Tamsulosin')

## 3-month lag

In [ ]:
#check lifelines version
import lifelines
lifelines.__version__

In [ ]:
tamsulosin_df['duration_year'].describe()

### Prepare data sets

In [ ]:
# 3 months lag
# finasteride data set
finasteride_3_months_lag = apply_lag_time(finasteride_df, 0.25)

# tamsulosin data set
tamsulosin_3_months_lag = apply_lag_time(tamsulosin_df, 0.25)

print(f'Total finasteride patients after 3 months lag: {len(finasteride_3_months_lag)}')
print(f'Total tamsulosin patients after 3 months lag: {len(tamsulosin_3_months_lag)}')

#merge data sets
combined_3_month_lag_df = pd.concat([tamsulosin_3_months_lag, finasteride_3_months_lag], axis = 0)

In [ ]:
summarize_variables_before_weighted(combined_3_month_lag_df, variables, grouped_variables)

In [ ]:
cumulative_incidence_and_km_plot(finasteride_3_months_lag, tamsulosin_3_months_lag, 'Finasteride', 'Tamsulosin', 'Cumulative Incidence for 3-month lag')

### Cox-model 

In [ ]:
#Unadjusted variables
Cox_proportional_model(tamsulosin_3_months_lag, finasteride_3_months_lag, 'Finasteride', 'Tamsulosin', unadjusted_variables)

In [ ]:
#Adjusted
Cox_proportional_model(tamsulosin_3_months_lag, finasteride_3_months_lag, 'finasteride','tamsulosin', variables)

### Propensity score

In [ ]:
#propensity score for 3 months lag
propensity_score_df_3_month_lag= estimate_propensity_scores(finasteride_3_months_lag, tamsulosin_3_months_lag)  

In [ ]:
summarize_variables_before_weighted(combined_3_month_lag_df,summary_variables,grouped_variables)

In [ ]:
#ipw 3 monhts lag
cox_ipw(propensity_score_df_3_month_lag, 'Finasteride', 'Tamsulosin', )

## 6-month lag

In [ ]:
# 6 months lag
# finasteride data set
finasteride_6_months_lag = apply_lag_time(finasteride_df, 0.5)

# tamsulosin data set
tamsulosin_6_months_lag = apply_lag_time(tamsulosin_df, 0.5)

print(f'Total finasteride patients after 6 months lag: {len(finasteride_6_months_lag)}')
print(f'Total tamsulosin patients after 6 months lag: {len(tamsulosin_6_months_lag)}')

#merge data sets
combined_3_month_lag_df = pd.concat([tamsulosin_6_months_lag, finasteride_6_months_lag], axis = 0)

In [ ]:
#Kaplan-Meier for 6 months lag
cumulative_incidence_and_km_plot(finasteride_6_months_lag, tamsulosin_6_months_lag, 'Finasteride', 'Tamsulosin', 'Cumulative Incidence for 6-month lag')

In [ ]:
# # 6 months lag
# # finasteride data set
# finasteride_6_months_lag = apply_lag_time(finasteride_df, 0.5)

# # tamsulosin data set
# tamsulosin_6_months_lag = apply_lag_time(tamsulosin_df, 0.5)

print(f'Total finasteride patients after 6 months lag: {len(finasteride_6_months_lag)}')
print(f'Total tamsulosin patients after 6 months lag: {len(tamsulosin_6_months_lag)}')

#merge data sets
combined_6_month_lag_df = pd.concat([tamsulosin_6_months_lag, finasteride_6_months_lag], axis = 0)

In [ ]:
#Unadjusted
Cox_proportional_model(tamsulosin_6_months_lag, finasteride_6_months_lag, 'Finasteride', 'Tamsulosin', unadjusted_variables)

In [ ]:
#Adjusted
Cox_proportional_model(tamsulosin_6_months_lag, finasteride_6_months_lag, 'tamsulosin', 'finasteride', variables)

### Propensity score

In [ ]:
#propensity score calculation
propensity_score_df_6_month_lag= estimate_propensity_scores(finasteride_6_months_lag, tamsulosin_6_months_lag)

In [ ]:
#IPTW for 6 months lag
cox_ipw(propensity_score_df_6_month_lag, 'Finasteride', 'Tamsulosin')

## 9-month lag

In [ ]:
#Prepare the data sets for 9-month lag
# finasteride data set
finasteride_9_months_lag = apply_lag_time(finasteride_df, 0.75)
#tamsulosin data set
tamsulosin_9_months_lag = apply_lag_time(tamsulosin_df, 0.75)
print(f'There are {len(finasteride_9_months_lag)} patients in the finasteride data set after 9 months lag')
print(f'There are {len(tamsulosin_9_months_lag)} patients in the tamsulosin data set after 9 months lag')

In [ ]:
#weighted incidence curve
cumulative_incidence_and_km_plot(finasteride_9_months_lag, tamsulosin_9_months_lag, 'Finasteride', 'Tamsulosin', 'Cumulative Incidence for 9-month lag')

In [ ]:
#Un-adjusted
Cox_proportional_model(tamsulosin_9_months_lag,finasteride_9_months_lag, 'Finasteride', 'Tamsulosin', unadjusted_variables)

In [ ]:
#IPTW for 9-month lag
propensity_score_df_9_month_lag= estimate_propensity_scores(finasteride_9_months_lag, tamsulosin_9_months_lag)
cox_ipw(propensity_score_df_9_month_lag, 'Finasteride', 'Tamsulosin')

### 1-year lag analysis

In [ ]:
# 12 months lag
# finasteride data set
finasteride_12_months_lag = apply_lag_time(finasteride_df, 1)

# tamsulosin data set
tamsulosin_12_months_lag = apply_lag_time(tamsulosin_df, 1)

#combined df for 12 months lag
combined_12_month_lag_df = pd.concat([finasteride_12_months_lag, tamsulosin_12_months_lag], axis = 0)

In [ ]:
#cummulative incidence
cumulative_incidence_and_km_plot(finasteride_12_months_lag, tamsulosin_12_months_lag, 'Finasteride', 'Tamsulosin', 'Cumulative Incidence for 12-month lag')

In [ ]:
#Crude HR for 1 year lag
Cox_proportional_model(tamsulosin_12_months_lag, finasteride_12_months_lag, 'tamsulosin', 'finasteride', unadjusted_variables)

In [ ]:
#calculate propensity score for 12 months lag
propensity_score_df_12_months_lag = estimate_propensity_scores(tamsulosin_12_months_lag, finasteride_12_months_lag)

#### IPTW for 12-month lag  

In [ ]:
#summary variables
summarize_variables_before_weighted(combined_12_month_lag_df, summary_variables, grouped_variables)

In [ ]:
#IPTW for 12-month lag
cox_ipw(propensity_score_df_12_months_lag, 'Finasteride', 'Tamsulosin')

### Strata

In [ ]:
#strata for primary analysis
run_cox_models_percentile_strata(finasteride_df, tamsulosin_df, 'Finasteride', 'Tamsulosin')

## Forest plot

In [ ]:
import matplotlib.pyplot as plt

# Study data
studies = ['Main analysis','3-month lag', '6-month lag', '9-month lag', '1-year lag', 'Lower stratum', 'Higher stratum']
mean = [1.27, 1.24, 1.26, 1.23, 1.15, 1.34, 1.20 ]
ci_lower = [0.95, 0.90, 0.90, 0.86, 0.77, 0.84, 0.82 ]
ci_upper = [1.72, 1.71, 1.76, 1.77, 1.72, 2.14, 1.76 ]

# Create a new figure and a subplot
fig, ax = plt.subplots(figsize=(8, 6), dpi = 300)

# Plot the mean values with larger markers
ax.scatter(mean, list(range(len(studies))), marker='s', color='red', s=100)

# Plot the confidence intervals
for i, study in enumerate(studies):
    ax.plot([ci_lower[i], ci_upper[i]], [i, i], color='black')

# Set the x-axis limits to be centered around 1
ax.set_xlim(0.5, 2)

# Remove the borders of the plot and y axis
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.get_yaxis().set_visible(False)

# Add a vertical line at x=1.0
ax.axvline(x=1.0, color='black', linestyle='--')

# Set the x-axis ticks
ax.set_xticks([0.5, 1.0, 2.0, 3.0])

# Remove labels and title
ax.set_xlabel('')
ax.set_title('')

# Invert the y-axis

ax.invert_yaxis()# Annotate the names of the studies at each point
for i, study in enumerate(studies):
    ax.text(0.4, i, study, ha='right')

plt.show()